<a href="https://colab.research.google.com/github/vibelleville/BloodCellDec22/blob/main/celldetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adaptation Sujet Academique Visage_detector [https://github.com/Bocbok/visage_detector] afin de detecter des cellules



1.   ère Etape: Detection de cellules principales [Barcelone]
2.   ème Etape: Detection de deux cellules [Dataset a définir]
3.   ème Etape: Detection sur une image de frottis



Dataset Barcelone
•	Cellule saines
•	17092 images classées par type de cellule
•	taille : 360 x 363 pixel
•	type : jpg

Description du Dataset https://data.mendeley.com/datasets/snkd93bnjr/1

Emplacement du Dataset https://data.mendeley.com/datasets/snkd93bnjr/1/files/2fc38728-2ae7-4a62-a857-032af82334c3/PBC_dataset_normal_DIB.zip


On importe le dataset dans l'espace de stockage de la session = content nb: on enlève les warnings pour plus de lisibilité

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!wget https://data.mendeley.com/datasets/snkd93bnjr/1/files/2fc38728-2ae7-4a62-a857-032af82334c3/PBC_dataset_normal_DIB.zip

--2023-08-07 10:35:31--  https://data.mendeley.com/datasets/snkd93bnjr/1/files/2fc38728-2ae7-4a62-a857-032af82334c3/PBC_dataset_normal_DIB.zip
Resolving data.mendeley.com (data.mendeley.com)... 162.159.133.86, 162.159.130.86
Connecting to data.mendeley.com (data.mendeley.com)|162.159.133.86|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /v1/datasets/snkd93bnjr/1/files/2fc38728-2ae7-4a62-a857-032af82334c3/PBC_dataset_normal_DIB.zip [following]
--2023-08-07 10:35:31--  https://data.mendeley.com/v1/datasets/snkd93bnjr/1/files/2fc38728-2ae7-4a62-a857-032af82334c3/PBC_dataset_normal_DIB.zip
Reusing existing connection to data.mendeley.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com/5a5a3b1a-0b1d-46bc-bff2-0420599d351b [following]
--2023-08-07 10:35:32--  https://prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com/5a5a3b1a-0b1d-46bc-bff2

On décompresse le dossier contenant le dataset nb: le chemin se trouve en clique droit sur fichier à gauche

In [ ]:
!unzip /content/PBC_dataset_normal_DIB.zip

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_292483.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_935950.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_890863.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_519370.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_215077.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_388154.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_968896.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_34345.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_349907.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_245683.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_785122.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_615525.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_714613.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_904757.jpg  

On importe les libraries basiques

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split



Le dossier Racine devient BARCELONA; on calcule du nombre d'images dans chaque dossier


In [ ]:
Barcelona= "/content/PBC_dataset_normal_DIB"
nb_images={}

for dir in os.listdir(Barcelona):
  nb_images[dir] = len(os.listdir(os.path.join(Barcelona,dir)))
nb_images.items()

dict_items([('erythroblast', 1551), ('lymphocyte', 1214), ('basophil', 1218), ('eosinophil', 3117), ('neutrophil', 3330), ('monocyte', 1420), ('ig', 2895), ('platelet', 2348)])

On crée un dataset  Datapath et Target

In [ ]:
dir_path = 'PBC_dataset_normal_DIB'

#Je crée 2 listes pour acceuillir les données
img_path = []
target = []

#creation de la boucle en allant chercher les premiers dossiers
for folder in os.listdir(path=dir_path):
  #on utilise join pour créer un nouveau path reprenant chaque nom de folder
  path = os.path.join(dir_path, folder)
  #on utilise ces nouveaux paths pour aller chercher les images
  for img_file in os.listdir(path=path):
    #on append la liste avec le chemin des images
    img_each_path = os.path.join(dir_path, folder, img_file)
    img_path.append(img_each_path)

    #travail sur la variable target : je divise la phrase au premier _
    split_string = img_file.split("_")
    #je récupère le premier élément et je le rajoute au folder name:
    target_name = folder + '_' + split_string[0]
    #je cet élement à la liste target
    target.append(target_name)

#je crée un dictionnaire pour le dataframe:
data = {
    'Path' : img_path,
    'target' : target
}

In [ ]:
#je crée le dataframe
df = pd.DataFrame(data)

In [ ]:
df.head()

,Path,target
0,PBC_dataset_normal_DIB/erythroblast/ERB_912846...,erythroblast_ERB
1,PBC_dataset_normal_DIB/erythroblast/ERB_972205...,erythroblast_ERB
2,PBC_dataset_normal_DIB/erythroblast/ERB_438112...,erythroblast_ERB
3,PBC_dataset_normal_DIB/erythroblast/ERB_823390...,erythroblast_ERB
4,PBC_dataset_normal_DIB/erythroblast/ERB_948793...,erythroblast_ERB


On rassemble les cellules sous un meme Label

In [ ]:
df=df.replace('eosinophil_EO','EOS')
df=df.replace('platelet_PLATELET','PLA')
df=df.replace(['neutrophil_SNE','neutrophil_BNE','neutrophil_NEUTROPHIL'],'NEU')
df=df.replace('erythroblast_ERB','ERY')
df=df.replace('monocyte_MO','MON')
df=df.replace('basophil_BA','BAS')
df=df.replace('lymphocyte_LY','LYM')
df=df.replace(['ig_MY','ig_MMY','ig_PMY','ig_IG'],'IG')
df.value_counts('target')

target
NEU               3329
EOS               3117
IG                2895
PLA               2348
ERY               1551
MON               1420
BAS               1218
LYM               1214
neutrophil_.DS       1
dtype: int64

On se separe des neutrophil_.DS [non repésentatif]

In [ ]:
df = df.drop(df.loc[df['target'] == 'neutrophil_.DS'].index)
df.value_counts('target')

target
NEU    3329
EOS    3117
IG     2895
PLA    2348
ERY    1551
MON    1420
BAS    1218
LYM    1214
dtype: int64

On garde les 125 premières images de chacunes des 8 classes


On renomme ces 1000 images de 1 à 1000, le paramètres k

In [ ]:
import os
from openpyxl import Workbook
from pathlib import Path


def rename_images(folder_path):
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif']  # Add more if needed

    image_files = [file for file in os.listdir(folder_path) if any(file.lower().endswith(ext) for ext in image_extensions)]

    workbook = Workbook()
    sheet = workbook.active
    sheet.title = "Image List"

    sheet["A1"] = "Original Name"
    sheet["B1"] = "New Name"

    for idx, image_file in enumerate(image_files):
        original_name = image_file
        new_name = f"{idx:04d}{Path(image_file).suffix}"  # Using zero-padding

        sheet.append([original_name, new_name])

        original_path = os.path.join(folder_path, original_name)
        new_path = os.path.join(folder_path, new_name)

        os.rename(original_path, new_path)

    excel_file_path = os.path.join(folder_path, "image_list.xlsx")
    workbook.save(excel_file_path)

if __name__ == "__main__":
    folder_path = "C:/Users/Bioregion/Desktop/DS/Projet/Datasets/PBC_dataset_1000_new"
    rename_images(folder_path)




Création d'un outil pour etiqueter les cellules dispo

    tracer un rectangle
    recuperer les coordonnées pixels definissant le rectangle

les 1000 images d'entrainement etiquettées avec  k, i, j, h, w
afin de construire le fichier label_train.txt

In [ ]:
import os
import shutil
import cv2 # install opencv_python

In [ ]:

from openpyxl import Workbook

class ImageROIApp:
    def __init__(self, image_files):
        self.image_files = image_files
        self.current_image_index = 0
        self.roi_list = []

        self.excel_file = "roi_coordinates.xlsx"
        self.create_excel_file()

        self.load_next_image()

    def create_excel_file(self):
        if not os.path.exists(self.excel_file):
            wb = Workbook()
            wb.save(self.excel_file)

    def load_next_image(self):
        if self.current_image_index < len(self.image_files):
            self.image_file = self.image_files[self.current_image_index]
            self.current_image = cv2.imread(self.image_file)
            self.copy_image = self.current_image.copy()  # Create a copy for drawing on
            self.roi_corners = []
            cv2.imshow("Image ROI Definition", self.current_image)
            cv2.setMouseCallback("Image ROI Definition", self.define_roi)
        else:
            print("ROIs defined for all images.")
            cv2.destroyAllWindows()

    def define_roi(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.roi_corners = [(x, y)]
        elif event == cv2.EVENT_MOUSEMOVE and len(self.roi_corners) == 1:
            self.copy_image = self.current_image.copy()
            x1, y1 = self.roi_corners[0]
            cv2.rectangle(self.copy_image, (x1, y1), (x, y), (0, 255, 0), 2)
            cv2.imshow("Image ROI Definition", self.copy_image)
        elif event == cv2.EVENT_LBUTTONUP:
            self.roi_corners.append((x, y))
            self.draw_roi_rectangle()
            self.save_roi_coordinates()

    def draw_roi_rectangle(self):
        if len(self.roi_corners) == 2:
            x1, y1 = self.roi_corners[0]
            x2, y2 = self.roi_corners[1]
            cv2.rectangle(self.copy_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.imshow("Image ROI Definition", self.copy_image)

    def save_roi_coordinates(self):
        self.roi_list.append({
            "image": os.path.basename(self.image_file),
            "x1": self.roi_corners[0][0],
            "y1": self.roi_corners[0][1],
            "x2": self.roi_corners[1][0],
            "y2": self.roi_corners[1][1]
        })

        wb = Workbook()
        ws = wb.active

        if not os.path.exists(self.excel_file):
            ws.append(["Image", "X1", "Y1", "X2", "Y2"])

        ws.append(["Image", "X1 ou Xmin", "Y1 ou min", "X2 ou max", "Y2 ou max", "k", "i", "j", "h", "w"])

        for roi_data in self.roi_list:
            ws.append([roi_data["image"], roi_data["x1"], roi_data["y1"], roi_data["x2"], roi_data["y2"], roi_data["image"][0:4], roi_data["y1"],roi_data["x1"],roi_data["y2"]-roi_data["y1"],roi_data["x2"]-roi_data["x1"]])

        wb.save(self.excel_file)

if __name__ == "__main__":
    image_files = [file for file in os.listdir() if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]

    if image_files:
        app = ImageROIApp(image_files)
        while True:
            key = cv2.waitKey(1) & 0xFF
            if key == 13:  # Enter key
                app.current_image_index += 1
                app.load_next_image()
                if app.current_image_index >= len(image_files):
                    break
            elif key == 27:  # Esc key
                cv2.destroyAllWindows()
                break
    else:
        print("No image files found in the current directory.")

No image files found in the current directory.


On positionne les images train dans un dossier train

In [ ]:
list = os.listdir('./train')
print(len(list))

998


On importe les libraries

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.optimizers import Adadelta, Adam, SGD, RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, PReLU, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import regularizers
#from util import *
from functools import partial
from itertools import combinations
from tqdm import tqdm
import glob
import tensorflow.keras.backend as K
import tensorflow as tf

On ajoute des classes et fonctions utiles

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from PIL import Image, ImageDraw
import itertools
from multiprocessing import Pool
from itertools import combinations
import tensorflow.keras.backend as K
import tensorflow as tf

REPRESENTATION_OF_1_IN_LOSS = 0.5

# Returns the IoU between 2 boxes
def IoU(box1, box2):
    _, i1, j1, h1, l1 = box1
    _, i2, j2, h2, l2 = box2

    l_intersection = min(j1 + l1, j2 + l2) - max(j1, j2)
    h_intersection = min(i1 + h1, i2 + h2) - max(i1, i2)
    # No overlap
    if l_intersection <= 0 or h_intersection <=0:
        return 0

    I = l_intersection * h_intersection

    U = l1 * h1 + l2 * h2 - I

    return I / U


def delete_overlaping_boxes(df):
    to_del = []

    for k in df.k.unique():
        frame = df[df.k == k].sort_values('score', ascending=False)

        for pair in combinations(range(len(frame)), r=2):
            b1 = frame.iloc[pair[0]]
            b2 = frame.iloc[pair[1]]

            I = IoU((_, b1.i, b1.j, b1.h, b1.w), (_, b2.i, b2.j, b2.h, b2.w))
            if I > 0.4:
                to_del.append(frame.iloc[pair[1]].name)

    return df.drop(np.unique(to_del))


def sliding_window(image, stride, wss):
    ret = []

    for ws in wss:
        sh = int(ws[0]*stride/100)
        sw = int(ws[1]*stride/100)
        for i in range(0, image.h-ws[0], sh):
            for j in range(0, image.w-ws[1], sw):
                if (i+ws[0] < image.h) and (j+ws[1] < image.w):
                    ret.append({'k':image.k,
                                    'i':i,
                                    'j':j,
                                    'h':ws[0],
                                    'w':ws[1]})
        for j in range(0, image.w-ws[1], sw):
            if (j+ws[1] < image.w):
                ret.append({'k':image.k,
                            'i':image.h-ws[0],
                            'j':j,
                            'h':ws[0],
                            'w':ws[1]})
        for i in range(0, image.w-ws[0], sh):
            if (i+ws[0] < image.h):
                ret.append({'k':image.k,
                            'i':i,
                            'j':image.w-ws[1],
                            'h':ws[0],
                            'w':ws[1]})

    return pd.DataFrame(ret)


def get_matching_boxes(true_box, box_list):
    print(len(box_list))
    temp = box_list.apply(lambda x: 1 if IoU(true_box, x) > 0.55 else 0, axis=1)
    return sum(temp)


def get_params_perf(params, imgs, labels):
    stride, ws = params

    box_list = []
    ret = []

    for idx, img in imgs.iterrows():
        boxes = sliding_window(img, stride, ws)
        box_list.append(boxes)
        ret.append(labels[labels.k == idx+1].apply(lambda x: get_matching_boxes(x, boxes), axis=1))


    box_list = pd.concat(box_list)
    ret = pd.concat(ret[:-1])

    stats = {'stride':stride,
            'ws':ws,
            'acc':sum(ret > 0)/len(ret),
            'mean':ret[ret != 0].mean(),
            'num_im':len(box_list)}

    return (ret, box_list, stats)


def crop_resize_save_img(im, x, box_labels, folder_size):
    iou = 0
    sub_folder, size = folder_size
    for _, box in box_labels.iterrows():
        n_iou = IoU(box, x)
        iou = round(n_iou, 2) if n_iou > iou else iou
    img = im.crop((x.j, x.i, x.j+x.w, x.i+x.h))
    img = img.resize((size,size))
    if iou > 0.2:
        img.save(f'train3{size}x{size}/{sub_folder}/1/{x.k}_{x.i}_{x.j}_{x.h}_{x.w}_{iou}.jpg')
    else:
        img.save(f'train3{size}x{size}/{sub_folder}/0/{x.k}_{x.i}_{x.j}_{x.h}_{x.w}_{iou}.jpg')

# im = {k, path, h, w}
def create_img_dataset(img, stride, window_size, box_labels, folder_size):
    _, img = img
    boxes = sliding_window(img, stride, window_size)
    with Image.open(img.path) as im:
        boxes.apply(lambda x: crop_resize_save_img(im, x, box_labels[box_labels.k == x.k], folder_size), axis=1)


# Determine the weight to apply to classes for imbalanced dataset
def class_weight(nb_1, nb_0, representation_of_1_in_loss=REPRESENTATION_OF_1_IN_LOSS):
    weight_0 = (1 - representation_of_1_in_loss) * (nb_0 + nb_1) / nb_0
    weight_1 = representation_of_1_in_loss * (nb_0 + nb_1) / nb_1
    return weight_0, weight_1

# Weighted binary cross entropy
def wbce( y_true, y_pred, sample_weight=None, weight1=1, weight0=1 ):
    y_true = K.clip(y_true, K.epsilon(), 1-K.epsilon())
    y_pred = K.clip(y_pred, K.epsilon(), 1-K.epsilon())
    logloss = -(y_true * K.log(y_pred) * weight1 + (1 - y_true) * K.log(1 - y_pred) * weight0 )
    return K.mean( logloss, axis=(0,1))

# Compute the best f1 score, precision, recall, prediction threshold
def f1(y_true, y_pred):
    df = pd.DataFrame(y_true, columns=['y_true'])
    df['y_pred'] = y_pred
    cleaner = df.duplicated('y_pred', keep='last')
    df = df.sort_values('y_pred', ascending=False)
    df['vp'] = df.y_true.cumsum()
    df['fp'] = (df.y_true == 0).cumsum()
    df['fn'] = sum(df.y_true) - df['vp']
    df['prec'] = df.vp / (df.vp + df.fp)
    df['rec'] = df.vp / (df.vp + df.fn)
    df = df.loc[~cleaner]
    df['f1'] = 2* (df.prec * df.rec) / (df.prec + df.rec)
    best = df['f1'].idxmax()
    return df.loc[best]

# Compute the best f2 score, precision, recall, prediction threshold
def f2(y_true, y_pred):
    df = pd.DataFrame(y_true, columns=['y_true'])
    df['y_pred'] = y_pred
    cleaner = df.duplicated('y_pred', keep='last')
    df = df.sort_values('y_pred', ascending=False)
    df['vp'] = df.y_true.cumsum()
    df['fp'] = (df.y_true == 0).cumsum()
    df['fn'] = sum(df.y_true) - df['vp']
    df['prec'] = df.vp / (df.vp + df.fp)
    df['rec'] = df.vp / (df.vp + df.fn)
    df = df.loc[~cleaner]
    df['f2'] = 5* (df.prec * df.rec) / (4*df.prec + df.rec)
    best = df['f2'].idxmax()
    return df.loc[best]

In [ ]:
def draw_boxes(path, boxes, true_boxes, color='red'):
    im = Image.open(path)
    draw = ImageDraw.Draw(im)
    if len(boxes) > 0:
        for idx, b in boxes.iterrows():
            draw.rectangle([b['j'], b['i'], b['j']+b['w'], b['i'] + b['h']], outline=color, width=2)

    if true_boxes != None:
        for idx, b in true_boxes.iterrows():
            draw.rectangle([b['j'], b['i'], b['j']+b['w'], b['i'] + b['h']], outline='white', width=2)

    return im

def local_IoU(box1, box2):

    l_intersection = min(box1.j + box1.w, box2.j + box2.w) - max(box1.j, box2.j)
    h_intersection = min(box1.i + box1.h, box2.i + box2.h) - max(box1.i, box2.i)
    # No overlap
    if l_intersection <= 0 or h_intersection <=0:
        return 0

    I = l_intersection * h_intersection

    U = box1.w * box1.h + box2.w * box2.h - I

    return I / U

In [ ]:
class visage_detector():

    def __init__(self, model, pred_t, stride=35, ws=[(100,85),(200,100),(250,212)]):
        self.model = model
        self.s = stride
        self.ws = ws
        self.pred_t = pred_t


    def get_box_groups(self, boxes, thresh=0.2):
        ret = boxes.copy()
        ret['area'] = boxes.w * boxes.h
        ret = ret.sort_values('area')

        groups = [[ret.iloc[0].name]]

        for idx, row in ret.iloc[1:].iterrows():
            create_new_group = True
            for g in groups:
                if (ret.loc[g].apply(lambda x: local_IoU(x.drop(['score','area']), row.drop(['score','area'])), axis=1)).max() > thresh:
                    g.append(idx)
                    create_new_group = False

            if create_new_group:
                groups.append([idx])

        return groups


    def merge_boxes(self, boxes_groups, boxes):
        merged_boxes = []
        for g in boxes_groups:
            b = boxes.loc[g]
            b = b.mean().astype(int)
            merged_boxes.append(b)

        return pd.DataFrame(merged_boxes)


    def get_merged_box_score(self, imgs, merged_boxes):

        to_pred = []

        merged_boxes.apply(lambda x: to_pred.append(np.asarray(Image.open(imgs.loc[x.k-1, 'path'])
                                                                          .crop((x.j, x.i, x.j+x.w, x.i+x.h))
                                                                          .resize((size, size)))*1./255), axis=1)

        print(len(to_pred), to_pred[0].shape)

        merged_boxes['score'] = self.model.predict(np.array(to_pred)).flatten()

        return merged_boxes



    def predict_one_image(self, img, merge=True):
        boxes = sliding_window(img, self.s, self.ws)

        to_pred = []

        with Image.open(img.path) as im:
            boxes.apply(lambda x: to_pred.append(
                np.asarray(im.crop((x.j, x.i, x.j+x.w, x.i+x.h))
                           .resize((size, size)))*1./255), axis=1)

        to_pred = np.array(to_pred)

        boxes['score'] = self.model.predict(to_pred).flatten()
        best_boxes = boxes[boxes.score > self.pred_t]

        if len(best_boxes)==0:
            return None

        if merge == False:
            return best_boxes
        else:
            merged_boxes = self.merge_boxes(self.get_box_groups(best_boxes), best_boxes).drop(columns='score')

            to_del = []

            if len(merged_boxes) > 1:
                for pair in combinations(range(len(merged_boxes)), r=2):
                    b1 = merged_boxes.iloc[pair[0]]
                    b2 = merged_boxes.iloc[pair[1]]

                    I = local_IoU(b1, b2)
                    if I > 0.4:
                        to_del.append(merged_boxes.iloc[pair[1]].name)

            return merged_boxes.drop(np.unique(to_del))


    def predict(self, df_imgs):
        predictions = []

        for idx, row in df_imgs.iterrows():
            predictions.append(self.predict_one_image(row))


        return pd.concat(predictions).reset_index().drop(columns='index')



    def evaluate(self, X, y):
        preds = self.predict(X)
        preds = self.get_merged_box_score(X, preds)
        preds = preds[preds.score > self.pred_t].drop(columns='score')

        tp = 0
        fp = 0
        fn = 0

        y_test = y.copy()
        y_test['counted'] = 0

        for idx, box in preds.iterrows():
            y_test = y_test[y_test.counted == 0]
            ious = y_test.drop(columns='counted').apply(lambda x: local_IoU(x, box), axis=1)
            if len(ious) != 0:
                if ious.max() >= 0.5:
                    tp += 1
                    y_test.loc[ious.idxmax(), 'counted'] = 1
                else:
                    print(box.k)
                    fp += 1


        fn = len(y) - tp

        p = tp / (tp + fp) if (tp+fp)!=0 else 0
        r = tp / (tp+fn) if (tp+fn)!=0 else 0

        f1 = 2*(p*r)/(p+r) if (p+r)!=0 else 0

        print(f'TP: {tp}, FP: {fp}, FN: {fn}')
        print(f'f1: {f1}, precision: {p}, recall: {r}')


        return f1, p, r


    def crop_resize(self, box, path):
        with Image.open(path) as im:
            return np.asarray(im.crop((box.j, box.i, box.j+box.w, box.i+box.h)).resize((size, size)))*1./255

In [ ]:
# Determine the weight to apply to classes for imbalanced dataset
def class_weight(nb_1, nb_0, representation_of_1_in_loss=REPRESENTATION_OF_1_IN_LOSS):
    weight_0 = (1 - representation_of_1_in_loss) * (nb_0 + nb_1) / nb_0
    weight_1 = representation_of_1_in_loss * (nb_0 + nb_1) / nb_1
    return weight_0, weight_1

# Weighted binary cross entropy
def wbce( y_true, y_pred, sample_weight=None, weight1=1, weight0=1 ):
    y_true = K.clip(y_true, K.epsilon(), 1-K.epsilon())
    y_pred = K.clip(y_pred, K.epsilon(), 1-K.epsilon())
    logloss = -(y_true * K.log(y_pred) * weight1 + (1 - y_true) * K.log(1 - y_pred) * weight0 )
    return K.mean( logloss, axis=(0,1))

# Compute the best f1 score, precision, recall, prediction threshold
def f1(y_true, y_pred):
    df = pd.DataFrame(y_true, columns=['y_true'])
    df['y_pred'] = y_pred
    cleaner = df.duplicated('y_pred', keep='last')
    df = df.sort_values('y_pred', ascending=False)
    df['vp'] = df.y_true.cumsum()
    df['fp'] = (df.y_true == 0).cumsum()
    df['fn'] = sum(df.y_true) - df['vp']
    df['prec'] = df.vp / (df.vp + df.fp)
    df['rec'] = df.vp / (df.vp + df.fn)
    df = df.loc[~cleaner]
    df['f1'] = 2* (df.prec * df.rec) / (df.prec + df.rec)
    best = df['f1'].idxmax()
    return df.loc[best]

# Compute the best f2 score, precision, recall, prediction threshold
def f2(y_true, y_pred):
    df = pd.DataFrame(y_true, columns=['y_true'])
    df['y_pred'] = y_pred
    cleaner = df.duplicated('y_pred', keep='last')
    df = df.sort_values('y_pred', ascending=False)
    df['vp'] = df.y_true.cumsum()
    df['fp'] = (df.y_true == 0).cumsum()
    df['fn'] = sum(df.y_true) - df['vp']
    df['prec'] = df.vp / (df.vp + df.fp)
    df['rec'] = df.vp / (df.vp + df.fn)
    df = df.loc[~cleaner]
    df['f2'] = 5* (df.prec * df.rec) / (4*df.prec + df.rec)
    best = df['f2'].idxmax()
    return df.loc[best]

Data analysis

In [ ]:
box_labels = pd.read_csv('label_train.txt', header=None, delimiter=' ',
                     names=['k', 'i', 'j', 'h', 'w'])
box_labels

In [ ]:

train_images = glob.glob('train\*')
df_images = pd.DataFrame(train_images, columns=['path'])
df_images['w'] = df_images.path.apply(lambda x: Image.open(x).size[0])
df_images['h'] = df_images.path.apply(lambda x: Image.open(x).size[1])
df_images['k'] = [x for x in range(1, 1001)]
df_images

In [ ]:
box_labels.h.hist()

In [ ]:
box_labels.w.hist()

In [ ]:
(box_labels.w / box_labels.h).hist()

Find best window size and stride

In [ ]:
f_part = partial(get_params_perf, imgs=df_images, labels=box_labels)

In [ ]:
%%time
if __name__ == '__main__':
    strides = [30,35,40]
    sizes = [[(100,85),(200,100),(250,212)]]

    for x in range(75, 400, 25):
        sizes.append([(x, int(x*0.50))])
        sizes.append([(x, int(x*0.65))])
        sizes.append([(x, int(x*0.85))])
        sizes.append([(x, x)])

    paramlist = list(itertools.product(strides, sizes))

    print(paramlist, len(paramlist))

    ret = []

    with Pool() as p:
        for m in tqdm(p.imap_unordered(f_part, paramlist)):
            ret.append(m)

In [ ]:
vp, box_list, stats= zip(*ret)

In [ ]:
box_list

In [ ]:
stats

In [ ]:
stats = pd.DataFrame(stats)
stats['mean/num_im'] = stats['mean']*100 / stats['num_im']
stats.sort_values('acc', ascending=False).head(30)

In [ ]:
vp = pd.DataFrame(vp)

In [ ]:
test = vp[33] + vp[40] + vp[13]
acc = sum(test > 0)/len(test)
n_im = sum([stats.iloc[i].num_im for i in [33, 40, 13]])
print(acc, n_im)

In [ ]:
# Use combinations to find best window size set
acc2 = []

for v1, v2, v3 in combinations(range(0, len(vp)), r=3):
    temp = vp[v1] + vp[v2] + vp[v3]
    if sum(temp>0)/len(temp) > 0.10:
        acc2.append(((v1, v2, v3), sum(temp>0)/len(temp)))

df = pd.DataFrame(acc2)
df

In [ ]:
df['n_im'] = df[0].apply(lambda x: sum([stats.iloc[x[i]].num_im for i in range(0,3)]))
df['n_im']

Dataset Creation

In [ ]:
from util import *
from functools import partial
from tqdm import tqdm

# Create the dataset. Need to run the func once for the train and once for validation


if __name__ == '__main__':

    folders = ['train', 'val']
    stride = 35
    window_size = [(100,85),(200,100),(250,212)]

    box_labels = pd.read_csv('label_train.txt', header=None, delimiter=' ',
                     names=['k', 'i', 'j', 'h', 'w'])

    part_create_train_dataset = partial(create_img_dataset,stride=stride, window_size=window_size ,
                                  box_labels=box_labels, folder_size=(folders[0],32))

    part_create_val_dataset = partial(create_img_dataset,stride=stride, window_size=window_size ,
                                  box_labels=box_labels, folder_size=(folders[1],32))


    train_images = glob.glob('train\*')
    df_images = pd.DataFrame(train_images, columns=['path'])
    df_images['w'] = df_images.path.apply(lambda x: Image.open(x).size[0])
    df_images['h'] = df_images.path.apply(lambda x: Image.open(x).size[1])
    df_images['k'] = [x for x in range(1, 1001)]
    df_images.head(20)


In [ ]:
 ret = []

    with Pool() as p:
        for m in tqdm(p.imap_unordered(part_create_train_dataset, df_images.iloc[:800].iterrows())):
            ret.append(m)

    ret = []

    with Pool() as p:
        for m in tqdm(p.imap_unordered(part_create_val_dataset, df_images.iloc[800:].iterrows())):
            ret.append(m)

Model

In [ ]:
ize = 32
input_shape = (size, size, 3)

def get_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    return model


Training

In [ ]:
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.optimizers import Adadelta, Adam, SGD, RMSprop
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from model import get_model
from callback import f1_class
from util import *
from functools import partial

#  global variables

batch_size = 512
epochs = 40
size = 32
nb1 = len(glob.glob('train32x32/train/1/*'))
nb0 = len(glob.glob('train32x32/train/0/*'))
input_shape = (size, size, 3)
steps_per_epochs = (len(glob.glob('train32x32/train/1/*'))+
                    len(glob.glob('train32x32/train/0/*')))/batch_size

validation_steps = (len(glob.glob('train32x32/val/1/*'))+
                    len(glob.glob('train32x32/val/0/*')))/batch_size

# Data generators

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'train32x32/train',
    target_size=(size,size),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    'train32x32/val',
    target_size=(size,size),
    batch_size=batch_size,
    class_mode='binary')


# Defined 2 generators without suffle for testing purpose
train_gen_no_shuffle = test_datagen.flow_from_directory(
    'train32x32/train',
    target_size=(size,size),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

val_gen_no_shuffle = test_datagen.flow_from_directory(
    'train32x32/val',
    target_size=(size,size),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

mc = ModelCheckpoint('model_weights/final04.{epoch:02d}-{loss:.2f}.hdf5', save_weights_only=True)
es = EarlyStopping(monitor='f1', patience=20, mode='max', restore_best_weights=True)
rl = ReduceLROnPlateau(monitor='loss', patience=6, factor=0.5, verbose=1)

model = get_model()
#model.load_weights('model_weights/final04.47-0.08.hdf5')

# Partials functions to pass to the model
w0, w1 = class_weight(nb1, nb0)
wb_part = partial(wbce, weight1=w1, weight0=w0)
f1_cb = f1_class(val_gen_no_shuffle)

model.compile(loss=wb_part,
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epochs,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[f1_cb,rl, mc],
    verbose=2)

y_val_true = val_gen_no_shuffle.labels
y_val_pred = model.predict_generator(val_gen_no_shuffle)
best = f1(y_val_true, y_val_pred.flatten())

print("seuil de prédiction optimal : ", best.y_pred)

In [ ]:
y_true = train_gen_no_shuffle.labels
y_pred = model.predict_generator(train_gen_no_shuffle)

F1 TRAINING SET

In [ ]:
f1(y_true, y_pred.flatten())

In [ ]:
f2(y_true, y_pred.flatten())

F1 VALIDATION SET

In [ ]:
%%time
y_val_true = val_gen_no_shuffle.labels
y_val_pred = model.predict_generator(val_gen_no_shuffle)
f1(y_val_true, y_val_pred.flatten())

In [ ]:
f2(y_val_true, y_val_pred.flatten())

GRAPHICS

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(f1_cb.f1s)
plt.title('Model f1')
plt.ylabel('f1')
plt.xlabel('Epoch')
plt.legend(['Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(f1_cb.re)
plt.title('Model recall')
plt.ylabel('recall')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(f1_cb.ps )
plt.title('Model precision')
plt.ylabel('precision')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

ON ENREGISTE LE RESUTAT DU MODEL SUR vd1

In [ ]:
%%time
vd1 = visage_detector(model, stride=30, ws=[(100,85),(200,100),(250,212)] , pred_t=0.980422)
vd1.evaluate(df_images, box_labels)

ON PREDIT SUR L IMAGE ayant k= 996 ET OBSERVE LE RESULTAT (VISUEL)

In [ ]:
img_k = 996
draw_boxes(df_images.loc[img_k-1].path, vd1.predict_one_image(df_images.loc[img_k-1], merge=True), None, 'red')

ON PREPARE LES DONNES TESTS
On rend : path 	w 	h 	k

In [ ]:
test_images = glob.glob('test\*')
test_images = pd.DataFrame(test_images, columns=['path'])
test_images['w'] = test_images.path.apply(lambda x: Image.open(x).size[0])
test_images['h'] = test_images.path.apply(lambda x: Image.open(x).size[1])
test_images['k'] = [x for x in range(1, 501)]
test_images

ON PREDIT SUR LES DONNEES TEST

In [ ]:
preds = vd1.predict(test_images)

In [ ]:
preds

In [ ]:
ON REND  k 	i 	j 	h 	w

In [ ]:
ON PREDIT SUR UNE IMAGE TEST  par exemple k=422

In [ ]:
img_test_k = 422
draw_boxes(test_images.loc[img_test_k-1].path, preds[preds.k == img_test_k], None, 'red')



---

